<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/06_24_22_GloVe_Run2_Conv1d_keras_kfold_300d_politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%%capture
!pip install wandb

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
import tensorflow as tf
import gensim.downloader as api

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from pathlib import Path

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

In [29]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
df = pd.read_csv(base_dir/"politifact_agg.csv", index_col=0)
df.head()

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"['3265439004', '3250621593', '3253922920', '32...",fake,NaN,[],24,3.218876,0,0.0
3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"['911971426571255810', '1036749614853103616', ...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


In [31]:
(df.title.str.len() == 0).any()

False

In [32]:
len(df), len(df.title.unique())

(894, 894)

In [33]:
news = df[['title', 'text', 'label']].copy()

print(news.isna().any())
news.count()

title    False
text      True
label    False
dtype: bool


title    894
text     741
label    894
dtype: int64

In [34]:
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Removing the special characters and not useful characters from URLs, data cleaning step. 


In [35]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
news['title']= news['title'].apply(denoise_text)

In [36]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.title)

In [37]:
d_wv = 50
glove = api.load(f'glove-wiki-gigaword-{d_wv}')

D_WV sets the length of the embedding vector. 

In [38]:
emb_matrix = np.zeros((max(tokenizer.word_index.values())+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)

In [39]:
len(tokenizer.word_index)

3310

In [48]:
def get_model(emb_matrix):
    model = tf.keras.Sequential([
        #alternatively we can use glove.get_keras_embedding(),
        tf.keras.layers.Embedding(emb_matrix.shape[0], output_dim=emb_matrix.shape[1], weights=[emb_matrix],trainable=False, mask_zero=True),
        # tf.keras.layers.Average(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(256, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=4),
        tf.keras.layers.LSTM(20, return_sequences=True),
        tf.keras.layers.LSTM(20),
        tf.keras.layers.Dropout(0.5),  
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dropout(0.4),  
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

Creating a model with LSTM layers to see how architecture affects the performance. 


In [49]:
model = get_model(emb_matrix)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 50)          165550    
                                                                 
 dropout_43 (Dropout)        (None, None, 50)          0         
                                                                 
 conv1d_14 (Conv1D)          (None, None, 256)         64256     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 256)        0         
 )                                                               
                                                                 
 lstm_5 (LSTM)               (None, None, 20)          22160     
                                                                 
 lstm_6 (LSTM)               (None, 20)                3280      
                                                     

## Titles

In [50]:
tokenized_titles = tokenizer.texts_to_sequences(news.title.tolist())
max_len = max(map(len, tokenized_titles))
X = sequence.pad_sequences(tokenized_titles, maxlen=max_len)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 25), (894,))

In [51]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [52]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model(emb_matrix)
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    probs = model.predict(X_test).squeeze()
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

In [53]:
params = {
    "epochs":100,
    "bs":128,
    # 'lr':1e-3
}

In [54]:
GROUP = f"glove{d_wv}-title-convnet"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)

Epoch 1/100
6/6 [==============================] - 4s 184ms/step - loss: 0.6853 - accuracy: 0.6145 - val_loss: 0.6656 - val_accuracy: 0.7472 - _timestamp: 1656071546.0000 - _runtime: 6.0000
Epoch 2/100
6/6 [==============================] - 0s 26ms/step - loss: 0.6489 - accuracy: 0.7221 - val_loss: 0.6064 - val_accuracy: 0.7978 - _timestamp: 1656071546.0000 - _runtime: 6.0000
Epoch 3/100
6/6 [==============================] - 0s 23ms/step - loss: 0.5704 - accuracy: 0.7919 - val_loss: 0.5187 - val_accuracy: 0.7978 - _timestamp: 1656071547.0000 - _runtime: 7.0000
Epoch 4/100
6/6 [==============================] - 0s 26ms/step - loss: 0.4777 - accuracy: 0.7961 - val_loss: 0.4750 - val_accuracy: 0.8146 - _timestamp: 1656071547.0000 - _runtime: 7.0000
Epoch 5/100
6/6 [==============================] - 0s 15ms/step - loss: 0.4294 - accuracy: 0.8366 - val_loss: 0.5090 - val_accuracy: 0.8146 - _timestamp: 1656071547.0000 - _runtime: 7.0000
Epoch 6/100
6/6 [==============================] - 0s 

accuracy,▁▄▅▆▆▇▇█▇███████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▇▅▄▃▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▂▅▄▆▄▅▅▆▄▅▆▇█▆▇▆▆▇▅▇▄▅▇███▅▇▆▅▇▇▆▆▇█▆▇▁▅
val_loss,▂▁▁▁▂▁▂▂▃▃▃▃▄▄▃▅▄▅▆▅▆▅▅▅▅▆▆▅▅▆▆▆▇▇▇▆▇█▅▅
accuracy,0.98883
best_epoch,3


## Fulltext

In [55]:
news['text'].fillna('', inplace=True)
news['text']= news['text'].apply(denoise_text)

news['fulltext'] = news['title'] + '\n' + news['text']

Using the full text of the news article to help with the prediction.


In [56]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.fulltext)

In [58]:
emb_matrix = np.zeros((len(tokenizer.word_index)+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)
print(emb_matrix.shape)

(33444, 50)


Creating an embedding matrix for the text too, so that it can be used as input. 


In [57]:
tokenized_texts = tokenizer.texts_to_sequences(news.fulltext.tolist())
max_len = 128
X = sequence.pad_sequences(tokenized_texts, maxlen=max_len)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 128), (894,))

In [59]:
params = {
    "epochs":100,
    "bs":128,
    # 'lr':1e-3
}

In [60]:
GROUP = f"glove{d_wv}-fulltext{max_len}-convnet"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)

Epoch 1/100
6/6 [==============================] - 4s 189ms/step - loss: 0.6851 - accuracy: 0.5670 - val_loss: 0.6446 - val_accuracy: 0.6798 - _timestamp: 1656071688.0000 - _runtime: 6.0000
Epoch 2/100
6/6 [==============================] - 0s 37ms/step - loss: 0.6135 - accuracy: 0.7151 - val_loss: 0.5491 - val_accuracy: 0.7416 - _timestamp: 1656071689.0000 - _runtime: 7.0000
Epoch 3/100
6/6 [==============================] - 0s 19ms/step - loss: 0.5271 - accuracy: 0.7556 - val_loss: 0.5583 - val_accuracy: 0.6966 - _timestamp: 1656071689.0000 - _runtime: 7.0000
Epoch 4/100
6/6 [==============================] - 0s 37ms/step - loss: 0.5367 - accuracy: 0.7514 - val_loss: 0.4717 - val_accuracy: 0.7921 - _timestamp: 1656071689.0000 - _runtime: 7.0000
Epoch 5/100
6/6 [==============================] - 0s 20ms/step - loss: 0.4511 - accuracy: 0.8142 - val_loss: 0.4997 - val_accuracy: 0.7753 - _timestamp: 1656071689.0000 - _runtime: 7.0000
Epoch 6/100
6/6 [==============================] - 0s 

accuracy,▁▄▅▆▆▇▇▇▇▇▇██████████████████▇██████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▆▆▅▄▃▃▃▃▃▂▂▁▁▁▁▁▂▁▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▅▅▅▇▇▆▄▅█▇█▇██▇▇████▆▇▇▇▇▇▆▄▇▇▇▇█▇█▇█▇
val_loss,▂▁▁▁▁▁▁▂▃▂▂▂▃▅▃▄▄▃▃▄▅▆█▃▄▄▅▆▄▆▄▅▅▅▅▅▅▅▅▅
accuracy,0.9986
best_epoch,6
